# AlexNet

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_alexnet(input_shape, num_classes):
    model = models.Sequential()

    # 1. Konvolüsyonel Katman
    model.add(layers.Conv2D(96, (5, 5), strides=(1, 1), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # 2. Konvolüsyonel Katman
    model.add(layers.Conv2D(256, (5, 5), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # 3. Konvolüsyonel Katman
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))

    # 4. Konvolüsyonel Katman
    model.add(layers.Conv2D(384, (3, 3), activation='relu', padding='same'))

    # 5. Konvolüsyonel Katman
    model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Düzleştirme Katmanı
    model.add(layers.Flatten())

    # Tam Bağlantılı Katmanlar
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model


In [ ]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# CIFAR-10 veri setini yükleyin ve yeniden boyutlandırın
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train, y_test = to_categorical(y_train), to_categorical(y_test)

# Modeli oluşturun
input_shape = (32, 32, 3)  # CIFAR-10 için giriş boyutu
num_classes = 10
model = create_alexnet(input_shape, num_classes)

# Modeli derleyin
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitin
history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_test, y_test))

# Modeli değerlendirin
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_acc}')


Epoch 1/10
782/782 [==============================] - 34s 33ms/step - loss: 1.7423 - accuracy: 0.3432 - val_loss: 1.4946 - val_accuracy: 0.4598
Epoch 2/10
782/782 [==============================] - 25s 32ms/step - loss: 1.3329 - accuracy: 0.5170 - val_loss: 1.2176 - val_accuracy: 0.5573
Epoch 3/10
782/782 [==============================] - 25s 32ms/step - loss: 1.1631 - accuracy: 0.5830 - val_loss: 1.1044 - val_accuracy: 0.6054
Epoch 4/10
782/782 [==============================] - 24s 31ms/step - loss: 1.0381 - accuracy: 0.6331 - val_loss: 1.0009 - val_accuracy: 0.6458
Epoch 5/10
782/782 [==============================] - 24s 31ms/step - loss: 0.9434 - accuracy: 0.6688 - val_loss: 1.0077 - val_accuracy: 0.6484
Epoch 6/10
782/782 [==============================] - 24s 31ms/step - loss: 0.8710 - accuracy: 0.6944 - val_loss: 0.9670 - val_accuracy: 0.6664
Epoch 7/10
782/782 [==============================] - 24s 31ms/step - loss: 0.8067 - accuracy: 0.7172 - val_loss: 1.0084 - val_accuracy:

# REsnet50

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, Add, Flatten, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

def identity_block(X, f, filters):
    F1, F2, F3 = filters
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Add shortcut value to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def convolutional_block(X, f, filters, s=2):
    F1, F2, F3 = filters
    X_shortcut = X

    # First component of main path
    X = Conv2D(F1, (1, 1), strides=(s, s))(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(F2, (f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(F3, (1, 1), strides=(1, 1))(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(F3, (1, 1), strides=(s, s))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Add shortcut to main path
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

def build_resnet50():
    input_shape = (224, 224, 3)
    X_input = Input(input_shape)

    # Zero-padding
    X = Conv2D(64, (7, 7), strides=(2, 2))(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # Average Pooling
    X = GlobalAveragePooling2D()(X)

    # Output layer
    X = Dense(1000, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

# Create the model
resnet50_model = build_resnet50()

# Print the model summary
resnet50_model.summary()


Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 109, 109, 64)         9472      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 109, 109, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 109, 109, 64)         0         ['batch_normalization[0

# DenseNet

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, Activation, MaxPooling2D, GlobalAveragePooling2D, Dense, Concatenate
from tensorflow.keras.models import Model

def dense_block(X, blocks, growth_rate):
    for _ in range(blocks):
        X = conv_block(X, growth_rate)
    return X

def conv_block(X, growth_rate):
    X1 = BatchNormalization()(X)
    X1 = Activation('relu')(X1)
    X1 = Conv2D(4 * growth_rate, (1, 1), use_bias=False)(X1)
    X1 = BatchNormalization()(X1)
    X1 = Activation('relu')(X1)
    X1 = Conv2D(growth_rate, (3, 3), padding='same', use_bias=False)(X1)
    X = Concatenate()([X, X1])
    return X

def transition_block(X, reduction):
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = Conv2D(int(X.shape[-1] * reduction), (1, 1), use_bias=False)(X)
    X = MaxPooling2D((2, 2), strides=(2, 2))(X)
    return X

def build_densenet121():
    input_shape = (224, 224, 3)
    growth_rate = 32

    X_input = Input(input_shape)

    # Initial Convolution
    X = Conv2D(64, (7, 7), strides=(2, 2), use_bias=False)(X_input)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Dense Block 1
    X = dense_block(X, 6, growth_rate)
    X = transition_block(X, 0.5)

    # Dense Block 2
    X = dense_block(X, 12, growth_rate)
    X = transition_block(X, 0.5)

    # Dense Block 3
    X = dense_block(X, 24, growth_rate)
    X = transition_block(X, 0.5)

    # Dense Block 4
    X = dense_block(X, 16, growth_rate)

    # Average Pooling
    X = GlobalAveragePooling2D()(X)

    # Output Layer
    X = Dense(1000, activation='softmax')(X)

    model = Model(inputs=X_input, outputs=X, name='DenseNet121')

    return model

# Create the model
densenet121_model = build_densenet121()

# Print the model summary
densenet121_model.summary()

Model: "DenseNet121"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_53 (Conv2D)          (None, 109, 109, 64)         9408      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_53 (Ba  (None, 109, 109, 64)         256       ['conv2d_53[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 activation_49 (Activation)  (None, 109, 109, 64)         0         ['batch_normalizatio

#vgg16

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def build_vgg16():
    input_layer = Input(shape=(224, 224, 3))

    # Block 1
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1')(input_layer)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')(x)

    # Fully connected layers
    x = Flatten(name='flatten')(x)
    x = Dense(4096, activation='relu', name='fc1')(x)
    x = Dense(4096, activation='relu', name='fc2')(x)
    x = Dense(1000, activation='softmax', name='predictions')(x)

    model = Model(inputs=input_layer, outputs=x, name='vgg16')

    return model

# Create the model
vgg16_model = build_vgg16()

# Print the model summary to see the layers
vgg16_model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     